In [1]:
import sys
sys.path.append(r'/home/nkrokhmal/Desktop/MyProjects/SergeyProject/umalat-project/prod')
from app.models import *
from app.models import Boiling as BoilingModel
from app.models import Termizator as TermizatorModel
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string


from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"/home/nkrokhmal/Desktop/MyProjects/SergeyProject/umalat-project/prod/data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()


In [4]:
type1 = CheeseType(
    name='вода',
    output=1000
)
type2 = CheeseType(
    name='соль',
    output=850
)
session.add(type1)
session.add(type2)
session.commit()

In [6]:
x = session.query(CheeseType).all()

In [8]:
water = [x for x in cheese_types if x.name == 'вода'][0]
salt = [x for x in cheese_types if x.name == 'соль'][0]

In [15]:
boilings = session.query(Boiling).all()
cheese_types = session.query(CheeseType).all()
water = [x for x in cheese_types if x.name == 'вода'][0]
salt = [x for x in cheese_types if x.name == 'соль'][0]
for boiling in boilings:
    if boiling.percent > 2.7:
        boiling.cheese_type = [water]
    else:
        boiling.cheese_type = [salt]
session.commit()

In [2]:
from app.utils.excel_client import *
import datetime
import os

In [3]:
def get_index(coordinate):
    xy = coordinate_from_string(coordinate) 
    col = column_index_from_string(xy[0])
    row = xy[1]
    return row, col

In [4]:
def parse(path):
    basename = os.path.basename(path)
    date_str = os.path.splitext(basename)[0]
    date = datetime.datetime.strptime(date_str, '%d.%m.%y')

    response = {'Date': date, 'WeekDay': date.weekday(), 'Boilings': []}
    
    skus = session.query(SKU).all()
    wb = openpyxl.load_workbook(filename=path)
    wb_data = openpyxl.load_workbook(filename=path, data_only=True)
    ws = wb['планирование суточное']
    ws_data = wb_data['планирование суточное']
    for i in range(1, 200):
        cell = ws.cell(i, 11).value
        if cell is not None:
            splited = re.split('=|-|/|\(|\)|\+|:', str(cell))
            splited = [get_index(x) for x in splited if x.startswith('F')]
            
            if len(splited) > 0:
                sku_names = [ws_data.cell(x[0], 3).value for x  in splited]
                boilings = [x.boilings[0] for x in skus if x.name in sku_names]
                if len(boilings) > 0:
                    if ws_data.cell(i, 12).value is not None:
                        tmp = {}
                        tmp['BoilingId'] = boilings[0].id
                        tmp['BoilingCount'] = ws_data.cell(i, 12).value
                        tmp['BoilingWeights'] = int(tmp['BoilingCount']) * [8000]
                        tmp_list = []
                        for s in splited:
                            sku_name = ws_data.cell(s[0], 3).value
                            sku_volume = ws_data.cell(s[0], 6).value
                            sku_ids = [x.id for x in skus if x.name == sku_name]

                            if len(sku_ids) > 0 and sku_volume is not None and sku_volume < 0:
                                tmp_list.append({sku_ids[0]: - sku_volume})
                        tmp['SKUVolumes'] = tmp_list
                        response['Boilings'].append(tmp)
    return response

In [5]:
parse('/home/nkrokhmal/Desktop/MyProjects/SergeyProject/umalat-project/prod/research/nkrokhmal/30.09.20.xlsx')

{'Date': datetime.datetime(2020, 9, 30, 0, 0),
 'WeekDay': 2,
 'Boilings': [{'BoilingId': 9,
   'BoilingCount': 1,
   'BoilingWeights': [8000],
   'SKUVolumes': [{12: 43.8}, {25: 426.375}]},
  {'BoilingId': 6,
   'BoilingCount': 1,
   'BoilingWeights': [8000],
   'SKUVolumes': [{23: 190.39999999999986},
    {27: 40.5},
    {21: 36},
    {26: 75.6},
    {22: 76.5},
    {24: 199}]},
  {'BoilingId': 6,
   'BoilingCount': 2,
   'BoilingWeights': [8000, 8000],
   'SKUVolumes': [{29: 1449.6}, {28: 45}, {32: 159}, {31: 116.4}]},
  {'BoilingId': 1,
   'BoilingCount': 9,
   'BoilingWeights': [8000, 8000, 8000, 8000, 8000, 8000, 8000, 8000, 8000],
   'SKUVolumes': [{38: 850}, {41: 524}, {39: 5900}]},
  {'BoilingId': 1,
   'BoilingCount': 1,
   'BoilingWeights': [8000],
   'SKUVolumes': [{34: 600}]},
  {'BoilingId': 1,
   'BoilingCount': 2,
   'BoilingWeights': [8000, 8000],
   'SKUVolumes': [{14: 1306}, {4: 318}]},
  {'BoilingId': 1,
   'BoilingCount': 1,
   'BoilingWeights': [8000],
   'SKUVolu

In [6]:
parse('/home/nkrokhmal/Downloads/2020 scheduling/2020/февраль/17.02.20.xlsx')

{'Date': datetime.datetime(2020, 2, 17, 0, 0),
 'WeekDay': 0,
 'Boilings': [{'BoilingId': 9,
   'BoilingCount': 0,
   'BoilingWeights': [],
   'SKUVolumes': []}]}